# CBOW Implementation of Word2Vec

This is part of the background research that I'm working on for [viberary.pizza](https://viberary.pizza/).

## Background 

[Word2vec](https://arxiv.org/abs/1301.3781) was a critical point in NLP work, building on previous work in dimensionality reduction in NLP such as tf-idf, topic modeling, and latent semantic analysis to reduce vocabulary sizes for computational complexity, and additionally, to add context by embedding similar words in the same latent space. 

As of 2022, it's almost been superceded by [transformers-based architectures](https://e2eml.school/transformers.html), but it's still worth understanding how it works in a historical context, as well as because there is a fair amount of it [in production in Spark](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.feature.Word2Vec.html).  

# Word2Vec Implementation

There are numerous word2vec implementations in libraries like Spark and Tensorflow. There is not an exact one in PyTorch, but following[this code](https://github.com/FraLotito/pytorch-continuous-bag-of-words/blob/master/cbow.py), as well as reading about the [architecture here](https://towardsdatascience.com/word2vec-with-pytorch-implementing-original-paper-2cd7040120b0) and [here](https://jalammar.github.io/illustrated-word2vec/),  I was able to implement and understand how it works under te covers


Original explanation in [PyTorch implementation](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html) is here. 

As a starting point, we take our raw data for input. 
Our [training set for Viberary is some sample the Goodreads input dataset](https://github.com/veekaybee/viberary#input-data-sample), 
which is a string of text containing the metadata for each unique book
id. 

For a single book id, it will contain the book description book title, etc. So a sample of a single book, will look like this


```
Raw text: All's Fairy in Love and War (Avalon: Web of Magic, #8) To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins 
have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. 
In an attempt to roust the pesky creatures, Kara falls through the portal, smack into the middle of a huge war.
Kara meets Queen Selinda, who appoints Kara as a Fairy Princess and assigns her an impossible task: 
to put an end to the war using her diplomatic skills.
```

This is initially stored as a Python string. 

Our final goal in learning a Word2Vec model with CBOW is, given an input phrase over a context window, to predict the word that's missing. The context window is how many words before and after the word we care about. So, given the phrase "Kara falls X the portal", we should be able to predict that the correct word is "through."

We do this in Word2Vec by continuously sampling from the raw text over the context window, where the context window around the word is the X variable and the word itself is the target variable. 

For the first example, "Kara falls the portal" is the context and "through" is the response variable. Then we shift the window by 1 word and generate another entry. This is the whole of the [continuous bag of words approach.](https://arxiv.org/pdf/1301.3781.pdf)

When we're first training the model, we pass these samples into the model and ask it to make a prediction on a single word given all these samples. The output is a vector of propabilities of the sample related to each word. We then compare that prediction to the actual label (I.e. for the sample "Kara falls X the portal" we KNOW the correct word is "through").

We compare the actual vector (i.e. where through = 1) to the probability vector, and the difference between the two is the loss. The parameters are passed to the model across multiple epochs and continuously updated until we minimize the loss, i.e. we get as close to the predicted word as possible. 

In the process of doing this prediction, we create a lookup table of words, or embeddings matrix, to their vector representations. It is these vectors that become our embeddings. Andiamo!

In [1]:
!pip install -q torch  # if you don't have it already


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# we need these bois
import torch
import torch.nn as nn

## Text Preparation

In [3]:
# First, we'll initialize our hyperparameters for the model:

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right - this is our context window
EMBEDDING_DIM = 100  # size of the embeddings matrix - we'll get to this in a bit

In [4]:
# Our tiny training dataset

raw_text = """To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. 
In an attempt to roust the pesky creatures, Kara falls through the portal, 
smack into the middle of a huge war. Kara meets Queen Selinda, who appoints 
Kara as a Fairy Princess and assigns her an impossible task: 
to put an end to the war using her diplomatic skills.""".split()

In [5]:
# Text preprocessing get only individual words
vocab = set(raw_text)  # dedup
vocab_size = len(vocab)

In [6]:
print(vocab)

{'opened', 'fallen', 'refuse', 'impending', 'bedroom', 'portal', 'has', 'attempt', 'Queen', 'meets', 'smack', 'discovers', 'goblins', 'realms', 'return', 'war.', 'have', 'impossible', "Kara's", 'the', 'closet', 'Selinda,', 'creatures,', 'war', 'To', 'fairy', 'Kara', 'put', 'assigns', 'Princess', 'an', 'skills.', 'astonishment,', 'to', 'into', 'a', 'pesky', 'falls', 'for', 'two', 'middle', 'portal,', 'huge', 'Fairy', 'task:', 'using', 'roust', 'her', 'that', 'through!', 'drafted', 'and', 'she', 'They', 'in', 'be', 'of', 'who', 'as', 'through', 'diplomatic', 'appoints', 'In', 'end'}


In [7]:
# we create simple mappings of word to an index of the word
word_to_ix = {word: ix for ix, word in enumerate(vocab)}
ix_to_word = {ix: word for ix, word in enumerate(vocab)}

In [8]:
print(word_to_ix)

{'opened': 0, 'fallen': 1, 'refuse': 2, 'impending': 3, 'bedroom': 4, 'portal': 5, 'has': 6, 'attempt': 7, 'Queen': 8, 'meets': 9, 'smack': 10, 'discovers': 11, 'goblins': 12, 'realms': 13, 'return': 14, 'war.': 15, 'have': 16, 'impossible': 17, "Kara's": 18, 'the': 19, 'closet': 20, 'Selinda,': 21, 'creatures,': 22, 'war': 23, 'To': 24, 'fairy': 25, 'Kara': 26, 'put': 27, 'assigns': 28, 'Princess': 29, 'an': 30, 'skills.': 31, 'astonishment,': 32, 'to': 33, 'into': 34, 'a': 35, 'pesky': 36, 'falls': 37, 'for': 38, 'two': 39, 'middle': 40, 'portal,': 41, 'huge': 42, 'Fairy': 43, 'task:': 44, 'using': 45, 'roust': 46, 'her': 47, 'that': 48, 'through!': 49, 'drafted': 50, 'and': 51, 'she': 52, 'They': 53, 'in': 54, 'be': 55, 'of': 56, 'who': 57, 'as': 58, 'through': 59, 'diplomatic': 60, 'appoints': 61, 'In': 62, 'end': 63}


In [9]:
# Creating our training data and context window


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = [raw_text[i - 2], raw_text[i - 1], raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

In [10]:
# We have our [input, input, input, input, target]
# based on the context window of +2 words -2 words
# you can see how we're building words close to each other now
print(*data[0:10], sep="\n")

(['To', "Kara's", 'she', 'discovers'], 'astonishment,')
(["Kara's", 'astonishment,', 'discovers', 'that'], 'she')
(['astonishment,', 'she', 'that', 'a'], 'discovers')
(['she', 'discovers', 'a', 'portal'], 'that')
(['discovers', 'that', 'portal', 'has'], 'a')
(['that', 'a', 'has', 'opened'], 'portal')
(['a', 'portal', 'opened', 'in'], 'has')
(['portal', 'has', 'in', 'her'], 'opened')
(['has', 'opened', 'her', 'bedroom'], 'in')
(['opened', 'in', 'bedroom', 'closet'], 'her')


## Model Set Up
# CBOW Architecture 

<img width="344" alt="Screen Shot 2023-02-14 at 3 48 16 PM" src="https://user-images.githubusercontent.com/3837836/218859716-495a0a6f-aed7-40aa-aba9-5c0f1949788c.png">

We have two layers in the CBOW implementation of Word2Vec: an input Embedding layer that maps each word to a space in the embedding dictionary, a hidden linear activation layer, and then the output layer that is the proportional probabilities [softmax](https://en.wikipedia.org/wiki/Softmax_function) of all the correct words given an input window. 

The critical part is the first part, creating the Embeddings lookup. 

First, we associate each word in the vocabulary with an index, aka `{'she': 0, 'middle': 1, 'put': 2`

Then, what we want to do is create an embeddings table, or matrix, that we will multiply with these indices to map each one to its correct place in relation to the other indices via a table lookup, based on how many vectors you'd like to represent the word. 

There is a [really good explanation](https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work/305032#305032) of how these are generated: 

``` 
For a given word, you create a one-hot vector based on its index and multiply it by the embeddings matrix, effectively replicating a lookup. For instance, for the word "soon" the index is 4, and the one-hot vector is [0, 0, 0, 0, 1, 0, 0]. If you multiply this (1, 7) matrix by the (7, 2) embeddings matrix you get the desired two-dimensional embedding, which in this case is [2.2, 1.4].
```

In [11]:
class CBOW(torch.nn.Module):
    def __init__(
        self, vocab_size, embedding_dim
    ):  # we pass in vocab_size and embedding_dim as hyperparams
        super(CBOW, self).__init__()

        # out: 1 x embedding_dim
        self.embeddings = nn.Embedding(
            vocab_size, embedding_dim
        )  # initialize an Embedding matrix based on our inputs
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()

        # out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim=-1)

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1, -1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        # Embeddings lookup of a single word once the Embeddings layer has been optimized
        return self.embeddings(word).view(1, -1)

In [12]:
# We initialize the model:

model = CBOW(vocab_size, EMBEDDING_DIM)

In [13]:
# then, we initialize the loss function
# (aka how close our predicted word is to the actual word and how we want to minimize it using the optimizer)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [14]:
# Training

# 50 to start with, no correct answer here
for epoch in range(50):
    # we start tracking how accurate our intial words are
    total_loss = 0

    # for the x, y in the training data:
    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)

        # we look at loss
        log_probs = model(context_vector)

        # we compare the loss from what the actual word is related to the probaility of the words
        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

    # optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # Log out some metrics to see if loss decreases
    print("end of epoch {} | loss {:2.3f}".format(epoch, total_loss))

end of epoch 0 | loss 336.137
end of epoch 1 | loss 324.402
end of epoch 2 | loss 313.337
end of epoch 3 | loss 302.782
end of epoch 4 | loss 292.611
end of epoch 5 | loss 282.781
end of epoch 6 | loss 273.231
end of epoch 7 | loss 263.993
end of epoch 8 | loss 254.978
end of epoch 9 | loss 246.182
end of epoch 10 | loss 237.563
end of epoch 11 | loss 229.070
end of epoch 12 | loss 220.699
end of epoch 13 | loss 212.436
end of epoch 14 | loss 204.293
end of epoch 15 | loss 196.253
end of epoch 16 | loss 188.313
end of epoch 17 | loss 180.481
end of epoch 18 | loss 172.724
end of epoch 19 | loss 165.083
end of epoch 20 | loss 157.594
end of epoch 21 | loss 150.262
end of epoch 22 | loss 143.088
end of epoch 23 | loss 136.076
end of epoch 24 | loss 129.240
end of epoch 25 | loss 122.594
end of epoch 26 | loss 116.136
end of epoch 27 | loss 109.904
end of epoch 28 | loss 103.904
end of epoch 29 | loss 98.137
end of epoch 30 | loss 92.612
end of epoch 31 | loss 87.343
end of epoch 32 | los

In [15]:
# Now, let's test to see if the model predicts the correct word using our initial input
context = ["Kara", "falls", "the", "portal"]
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

In [16]:
print(f'Raw text: {" ".join(raw_text)}\n')
print(f"Context: {context}\n")
print(f"Prediction: {ix_to_word[torch.argmax(a[0]).item()]}")

Raw text: To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. In an attempt to roust the pesky creatures, Kara falls through the portal, smack into the middle of a huge war. Kara meets Queen Selinda, who appoints Kara as a Fairy Princess and assigns her an impossible task: to put an end to the war using her diplomatic skills.

Context: ['Kara', 'falls', 'the', 'portal']

Prediction: creatures,


In [17]:
# Now let's get what we care about, which is the embeddings!
print(f"Getting vectors for a sequence:\n", model.embeddings(torch.LongTensor([1, 2, 3])))

Getting vectors for a sequence:
 tensor([[-1.3122e+00, -5.9373e-03,  5.9570e-01, -1.8307e-01,  8.8574e-01,
          3.7438e-01, -2.0238e-01,  2.2714e+00,  7.4274e-01,  1.4457e+00,
         -1.0922e-01, -7.2746e-01,  1.8884e+00,  1.5903e+00, -2.2890e-01,
          2.7072e-01, -4.3012e-01, -8.8410e-01,  1.0167e+00,  3.9204e-01,
         -1.4966e-02, -6.9684e-01,  1.2691e+00, -7.9018e-01, -4.1049e-01,
          6.9828e-01, -5.7492e-01,  1.3252e+00,  1.6870e-01, -1.8804e+00,
         -6.9079e-01, -1.3518e+00,  1.8567e+00, -1.0202e+00,  6.8851e-01,
          1.0466e+00, -1.5274e+00,  6.7465e-01,  5.1905e-01, -4.9612e-01,
          9.1026e-01, -7.6268e-01,  6.6666e-01,  2.2071e+00, -9.0811e-01,
         -2.4226e-01, -1.0076e+00,  7.9627e-01,  7.2852e-01, -6.6621e-01,
          6.2031e-01, -3.4981e-01, -3.1215e-02,  5.9000e-01, -5.2303e-01,
         -1.1391e-01,  2.5255e+00,  5.1060e-01,  1.9733e+00,  3.3426e-01,
          9.6696e-01, -1.7444e+00, -6.5390e-01, -3.9971e-01,  1.7169e-01,
     

In [18]:
print(
    "Getting weights:\n", model.embeddings.weight.data[1]
)  # we can get the entire matrix this way

Getting weights:
 tensor([-1.3122, -0.0059,  0.5957, -0.1831,  0.8857,  0.3744, -0.2024,  2.2714,
         0.7427,  1.4457, -0.1092, -0.7275,  1.8884,  1.5903, -0.2289,  0.2707,
        -0.4301, -0.8841,  1.0167,  0.3920, -0.0150, -0.6968,  1.2691, -0.7902,
        -0.4105,  0.6983, -0.5749,  1.3252,  0.1687, -1.8804, -0.6908, -1.3518,
         1.8567, -1.0202,  0.6885,  1.0466, -1.5274,  0.6746,  0.5190, -0.4961,
         0.9103, -0.7627,  0.6667,  2.2071, -0.9081, -0.2423, -1.0076,  0.7963,
         0.7285, -0.6662,  0.6203, -0.3498, -0.0312,  0.5900, -0.5230, -0.1139,
         2.5255,  0.5106,  1.9733,  0.3343,  0.9670, -1.7444, -0.6539, -0.3997,
         0.1717, -0.4201, -0.1215,  0.4620, -0.3666, -1.2402,  0.2124,  0.5813,
        -1.8909, -0.8708,  0.2327,  1.6975, -1.9238,  0.2262, -0.4582, -1.6192,
         0.4325, -0.5825,  0.9860, -0.5981, -0.2162,  0.3044,  0.2164, -1.1491,
        -0.9855, -0.8305,  0.0323, -1.1100,  0.1646,  0.7922,  0.5116,  0.6018,
         1.1963,  0.57

In [19]:
# And, what we actually care about is being able to look up individual words with their embeddings:
torch.set_printoptions(threshold=10_000)
print(f"Embedding for Kara: {model.embeddings.weight[word_to_ix['Kara']]}")

Embedding for Kara: tensor([ 1.0888, -1.1063,  0.8696, -0.3632, -0.9006,  0.7940, -1.1138,  2.1278,
        -0.1689, -0.3420, -1.3995,  0.0601, -2.9023, -0.5998,  1.1930, -0.7473,
         1.5664,  1.1403,  0.3928,  0.3714,  0.3632,  0.1330, -1.7100,  0.0202,
         2.0143, -0.3630,  1.2034,  1.0377,  1.2923, -1.1941,  0.3272,  2.8864,
        -1.4247,  0.4827,  1.4733, -1.1051,  0.7728, -0.3916, -0.2147, -0.2876,
        -0.9865,  0.1281, -1.9323, -0.9826, -1.2451, -1.1102, -1.0630,  0.6419,
        -1.5088,  1.0152,  0.4989, -0.0460,  0.2530, -0.2517, -1.1437, -0.3275,
        -1.1290,  1.2488,  0.9396, -0.6129, -1.5827,  0.0347,  1.4194,  0.1029,
        -0.8398, -1.2183, -1.9156, -0.1418, -0.5877, -0.8353, -2.2197, -1.4046,
         1.4064,  0.6054,  0.7366,  0.9687, -0.4782, -1.3862,  0.2172, -0.6948,
         1.0114,  0.0751, -1.7392,  1.6850,  1.2450, -0.6311,  0.3511, -0.5585,
         0.3689, -0.5361,  1.6833, -1.1443, -0.5681, -0.4838,  0.2609, -1.9220,
         0.4201,  0.

In [20]:
# This way, when we create our second tower of book words, we know which ones are likely related to a given book